### Import dependencies

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization

### Load CSV data

In [ ]:
load_dotenv()

DATA_DIR = os.getenv('DATA_DIR')

In [ ]:
def load_train_df() -> None:
    return pd.read_csv(f"{DATA_DIR}/train.csv", dtype={
        "img": int, # Image ID
        "x": int, # X1 coordinate of the top-left corner of the bounding box
        "y": int, # Y1 coordinate of the top-left corner of the bounding box
        "w": int, # X2 of the bounding box
        "h": int, # Y2 of the bounding box
        "rot": int, # Whether the image shows a rotten pear (1) or not (0)
    }, index_col=0).rename(columns={
        "img": "image_id",
        "x": "x1",
        "y": "y1",
        "w": "x2",
        "h": "y2",
        "rot": "is_rotten",
    })

In [ ]:
def load_test_df() -> None:
    return pd.read_csv(f"{DATA_DIR}/test.csv", dtype={
        "img": int, # Image ID
        "x": int, # X1 coordinate of the top-left corner of the bounding box
        "y": int, # Y1 coordinate of the top-left corner of the bounding box
        "w": int, # X2 of the bounding box
        "h": int, # Y2 of the bounding box
    }, index_col=0).rename(columns={
        "img": "image_id",
        "x": "x1",
        "y": "y1",
        "w": "x2",
        "h": "y2",
    })

In [ ]:
train_df = load_train_df()
test_df = load_test_df()

### Load images

In [ ]:
def safe_create_directory(directory: str) -> None:
    try:
        os.makedirs(directory, exist_ok=True)
    except OSError as e:
        print(f"Error creating directory {directory}: {e}")

In [ ]:
def get_image_path(image_id: int, directory: str) -> str:
    return f"{directory}{image_id:08d}.png"

In [ ]:
def load_image(image_id: int, directory: str) -> np.ndarray:
    image_path = get_image_path(image_id, directory)
    image = cv2.imread(image_path)

    return image

In [ ]:
def determine_training_subdirectory(row: pd.Series) -> str:
    if "is_rotten" not in row:
        return ""
    
    return "rotten/" if row["is_rotten"] == 1 else "not_rotten/"

In [ ]:
def get_cropped_image_path(row: pd.Series, directory: str) -> str:
    subdirectory = determine_training_subdirectory(row)
    return f"{directory}{subdirectory}{row.name}.png"

In [ ]:
def is_dir_empty(dir: str) -> bool:
    safe_create_directory(dir)

    return len(os.listdir(dir)) == 0

In [ ]:
def crop_image(image: np.ndarray, row: pd.Series) -> np.ndarray:
    left_x, top_y, right_x, bottom_y = row[["x1", "y1", "x2", "y2"]]

    cropped_image = image[top_y:bottom_y, left_x:right_x]

    return cropped_image

In [ ]:
def crop_all_images(df: pd.DataFrame, raw_directory: str, cropped_directory: str) -> None:   
    if not is_dir_empty(cropped_directory):
        print(f"The {cropped_directory} directory is not empty. Skipping.")
        return

    # Load all images one by one
    for image_id in df["image_id"].unique():
        image = load_image(image_id, raw_directory)

        # Find all the rows with the current image ID
        rows = df[df["image_id"] == image_id]

        # Crop the image for each row
        for _, row in rows.iterrows():
            cropped_image = crop_image(image, row)
            cropped_image_path = get_cropped_image_path(row, cropped_directory)
            # Save the cropped image
            cv2.imwrite(cropped_image_path, cropped_image)

In [ ]:
def add_dimensions(df: pd.DataFrame) -> None:
    df["width"] = df["x2"] - df["x1"]
    df["height"] = df["y2"] - df["y1"]

In [ ]:
# Define the directories for raw and cropped images
IMAGE_DATA_DIRECTORIES = {
    "raw": {
        "train": f"{DATA_DIR}/raw/all_images/",
        "test": f"{DATA_DIR}/raw/all_images/",
    },
    "cropped": {
        "train": f"{DATA_DIR}/cropped/train_images/",
        "test": f"{DATA_DIR}/cropped/test_images/",
    },
}

for directory in IMAGE_DATA_DIRECTORIES["raw"].values():
    safe_create_directory(directory)

for directory in IMAGE_DATA_DIRECTORIES["cropped"].values():
    safe_create_directory(directory)

In [ ]:
crop_all_images(train_df, IMAGE_DATA_DIRECTORIES["raw"]["train"], IMAGE_DATA_DIRECTORIES["cropped"]["train"])
crop_all_images(test_df, IMAGE_DATA_DIRECTORIES["raw"]["test"], IMAGE_DATA_DIRECTORIES["cropped"]["test"])

In [ ]:
add_dimensions(train_df)